In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
from src.summarize_text import SpeechSummaryProcessor

In [3]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Device cuda:0
Beginning Transcribing Process...


In [4]:
text

'1: After eight hours, I went back to put the map\n2: It was now I seemed to write the essay and the rain now I go to bed'